In [1]:
%load_ext autoreload
%autoreload 2
# interface to external tools
import admix
import numpy as np
import pandas as pd

In [4]:
dset = admix.data.load_lab_dataset("ukb_eur_afr")
snp = dset["snp"].values
new_pos = admix.misc.lift_over(chrom_pos=np.vstack([dset["chrom@snp"].values, dset["position@snp"].values]).T, chain="hg38->hg19")
df_new = pd.DataFrame({"CHR": dset["chrom@snp"].values, "POS": new_pos})
bim = admix.misc.plink_read_bim([f"/u/project/sriram/ukbiobank/data/geno/cal/{i_chr}.bim" for i_chr in range(1, 23)])
df_merged = df_new.merge(bim, on=["CHR", "POS"])

In [5]:
np.savetxt("./snp_list.txt", df_merged.SNP.values, fmt="%s")

# Subset
```bash
for i_chr in $(seq 22); do
    plink \
        --bfile /u/project/sriram/ukbiobank/data/geno/cal/${i_chr} \
        --extract snp_list.txt \
        --keep-allele-order \
        --make-bed \
        --out plink_all/chr${i_chr}
done
```

In [3]:
admix.misc.plink_merge(bfiles=[f"plink/all/chr{i_chr}" for i_chr in range(1, 23)], out="plink/all/merged")

In [38]:
df_eur = admix.misc.plink_read_fam("/u/project/sriram/ukbiobank/data/geno/cal/filter4.fam")
df_eur.to_csv("./plink/eur.fam", index=False, header=False, sep='\t')
df_admix = pd.DataFrame([i.split("_") for i in dset.indiv.values]).astype(int)
df_admix.to_csv("./plink/admix.fam", index=False, header=False, sep='\t')
df_all = admix.misc.plink_read_fam("./plink/all/merged.fam")

# Subset admixed individuals
```bash
for pop in eur admix; do
    mkdir -p plink/${pop}
    plink \
        --bfile plink/all/merged \
        --keep-fam plink/${pop}.fam \
        --keep-allele-order \
        --make-bed \
        --out plink/${pop}/merged
done
```